In [ ]:
!pip -q install keras "tensorflow==2.15.1" "onnx" "seaborn" "onnxruntime"

In [ ]:
import keras
import pickle
import pandas as pd
import numpy as np

In [ ]:
PRED_COUNTRY = "output-8"

In [ ]:
keras_model = keras.saving.load_model("trained_model.keras")

with open('scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

In [ ]:
# We pick the properties of our favorite song
song_properties = pd.read_parquet('../99-data_prep/song_properties.parquet')
favorite_song = song_properties.loc[song_properties["name"]=="Not Like Us"]
favorite_song

In [ ]:
song_properties = favorite_song[['is_explicit', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
song_properties.T

In [ ]:
scaled_feature = scaler.transform(song_properties)[0]
scaled_feature

In [ ]:
feature_values = {
    "is_explicit": scaled_feature[0],
    "duration_ms": scaled_feature[1],
    "danceability": scaled_feature[2],
    "energy": scaled_feature[3],
    "key": scaled_feature[4],
    "loudness": scaled_feature[5],
    "mode": scaled_feature[6],
    "speechiness": scaled_feature[7],
    "acousticness": scaled_feature[8],
    "instrumentalness": scaled_feature[9],
    "liveness": scaled_feature[10],
    "valence": scaled_feature[11],
    "tempo": scaled_feature[12]
}

feature_df = pd.DataFrame([feature_values])
feature_df.T

In [ ]:
output_names = []
for i in range(72):
    output_names.append(f"output-{i}")

In [ ]:
print(f"Predicted probability for our selected country is {keras_model.predict(feature_df)[0][output_names.index(PRED_COUNTRY)]}")

In [ ]:
def pred(x):
    pred = keras_model.predict(x)
    pred = np.squeeze(pred)
    pred = {output_names[i]: pred[i] for i in range(pred.shape[0])}
    print(f"Predicted probability is: {pred[PRED_COUNTRY]}")
    if pred[PRED_COUNTRY] >= 0.3:
        pred = {PRED_COUNTRY: True}
    else:
        pred = {PRED_COUNTRY: False}
    return pd.DataFrame([pred])

In [ ]:
from trustyai.model import Model

model = Model(pred, output_names=[PRED_COUNTRY])

In [ ]:
from trustyai.model.domain import feature_domain
_domains = {
        "is_explicit": (0.0, 1.0),
        "duration_ms": (0.0, 1.0),
        "danceability": (0.0, 1.0),
        "energy": (0.0, 1.0),
        "key": (0.0, 1.0),
        "loudness": (0.0, 1.0),
        "mode": (0.0, 1.0),
        "speechiness": (0.0, 1.0),
        "acousticness": (0.0, 1.0),
        "instrumentalness": (0.0, 1.0),
        "liveness": (0.0, 1.0),
        "valence": (0.0, 1.0),
        "tempo": (0.0, 1.0)
}
domains = {key: None for key  in feature_values.keys()}

for key in  _domains.keys():
        domains[key] = feature_domain(_domains[key])

domains = list(domains.values())

In [ ]:
from trustyai.model import output
goal = [output(name=PRED_COUNTRY, dtype="bool", value=True)]

In [ ]:
from trustyai.explainers import CounterfactualExplainer

explainer = CounterfactualExplainer(steps=50)
explanation = explainer.explain(inputs=feature_df, goal=goal, model=model, feature_domains=domains)

In [ ]:
model(explanation.proposed_features_dataframe)

In [ ]:
explanation.as_dataframe()

In [ ]:
df = explanation.as_dataframe()
df[df.difference != 0.0]

In [ ]:
explanation.plot()